In [98]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

In [82]:
def cars24_search(search_type,location):
    """Generate a URL for search term with location"""
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    search_type = search_type.replace(' ','%20')
    link = f'https://www.cars24.com/buy-used-car?sort=P&search={search_type}&storeCityId=2378&pinId={LOC[location]}'
    
    return link


In [20]:
def cars24_scrapped_data():
    """ 
    Function to scrap datas from cars 24.
    
    """
      
    print("SCRAPPING THE REQUIRED DETAILS")
    driver = webdriver.Chrome(r"C:/Users/swati/Downloads/chromedriver_win32 (1)/chromedriver.exe")    
    driver.maximize_window()
    
    LOC = {'Delhi': '110001', 'Noida': '201301', 'Gurgoan': '122001', 'Mumbai' : '400001', 'Pune': '411001', 
           'Banglore': '560001','Hyderabad': '500001', 'Chennai': '600001', 'Kolkata': '700001', 'Ahmedabad': '380001' }
    
    location = [keys for keys in LOC.keys()]
    
    Car_type= ['SUV', 'Sedan', 'Hatchback','Coupe','Minivan' ,'Luxury Sedan', 'Luxury SUV']
    
    URL = [cars24_search(j,i) for i in location for j in Car_type]
    
    LOCATION = []
    MNF_YEAR = []
    BRAND = []
    MODEL = []
    VARIANT = []
    TRANSMISSION=[]
    driven_KM = []
    NO_OF_owners = []
    fuel_type = []
    PRICE = []


    for i in URL:
        driver.get(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class' : '_1l4fi'})

        brand_model_name = []
        variant_transmission_name = []
        km_owner_fuel= []
        price = []
    
        for i in results:
            brand_model_name.append(i.find('h2', {'class': "_3FpCg"}).text)
            variant_transmission_name.append(i.find('p', {'class': "cvakB"}).text)
            km_owner_fuel.append(i.find('ul', {'class': "bVR0c"},'li').text)
            price.append(i.find('div', {'class': "_7udZZ"}).text)
            LOCATION.append(driver.find_element_by_xpath("//p[@class='_1V4-O']").text)
    
    
        year = [i.split(' ',2)[0] for i in brand_model_name]
        brand = [i.split(' ',2)[1] for i in brand_model_name]
        #D1 = [i.split(' ',2)[-1] for i in brand_model_name]
        name= [i.split(' ',2)[-1] for i in brand_model_name]

        for i in price:
            PRICE.append(i)

        for i in year:
            MNF_YEAR.append(i)
        for i in brand:
            BRAND.append(i)
    
        #D2 = [i.rsplit(' ',1)[0] for i in variant_transmission_name]
        variant = [i.rsplit(' ',1)[0] for i in variant_transmission_name]
        transmission = [i.rsplit(' ',1)[1] for i in variant_transmission_name]
        for i in variant:
            VARIANT.append(i)
    
    
        for i in name:
            MODEL.append(i)
        for i in transmission:
            TRANSMISSION.append(i)
    
        dummy1 = [i.split('km')[-1]for i in km_owner_fuel]
        kms = [i.split('km')[0] for i in km_owner_fuel]
        kms = [i+'km' for i in kms]

        owners = [i.split('Owner')[0] for i in dummy1]
        owners = [i+'Owner' for i in owners]
        fuel = [i.split('Owner')[-1] for i in dummy1]

        for i in kms:
            driven_KM.append(i)

        for i in owners:
            NO_OF_owners.append(i)

        for i in fuel:
            fuel_type.append(i)

    driver.close()
        
    used_cars_data = pd.DataFrame({"LOCATION" : LOCATION,
                              "MNF_YEAR" : MNF_YEAR,
                              "BRAND" : BRAND,
                              "MODEL" : MODEL,
                              "VARIANT" : VARIANT,
                             "TRANSMISSION": TRANSMISSION,
                              "DRIVEN_KM" : driven_KM,
                              "FUELTYPE" : fuel_type,
                              "NO_OF_OWNERS" : NO_OF_owners,
                              "PRICE" : PRICE
                             })
    
    used_cars_data.to_csv("used_cars-data.csv",index=False) #Creating CSV
    
    return used_cars_data

car_scrap_data = cars24_scrapped_data()
car_scrap_data

SCRAPPING THE REQUIRED DETAILS


,LOCATION,MNF_YEAR,BRAND,MODEL,VARIANT,TRANSMISSION,DRIVEN_KM,FUELTYPE,NO_OF_OWNERS,PRICE
0,New Delhi,2016,Maruti,Vitara Brezza,ZDI,Manual,"1,01,529 km",Diesel,1st Owner,"₹6,55,999"
1,New Delhi,2016,Maruti,Vitara Brezza,VDI OPT,Manual,"59,773 km",Diesel,1st Owner,"₹6,66,399"
2,New Delhi,2017,Maruti,Vitara Brezza,VDI OPT,Manual,"35,231 km",Diesel,1st Owner,"₹6,84,399"
3,New Delhi,2013,Ford,Ecosport,1.5TITANIUM TDCI,Manual,"80,946 km",Diesel,2nd Owner,"₹4,09,899"
4,New Delhi,2016,Ford,Ecosport,1.5 TITANIUM TI VCT,Manual,"16,212 km",Petrol,1st Owner,"₹7,41,699"
...,...,...,...,...,...,...,...,...,...,...
8663,Ahmedabad,2019,Maruti,Eeco,5 STR WITH AC PLUSHTR,Manual,"26,880 km",Petrol,1st Owner,"₹4,59,599"
8664,Ahmedabad,2019,Maruti,Eeco,5 STR WITH AC PLUSHTR,Manual,"11,882 km",Petrol,1st Owner,"₹4,61,999"
8665,Ahmedabad,2017,Ford,Ecosport,1.5TITANIUM TDCI,Manual,"71,900 km",Diesel,2nd Owner,"₹6,16,799"
8666,Ahmedabad,2018,Maruti,Vitara Brezza,VDI,Manual,"27,460 km",Diesel,1st Owner,"₹8,16,199"


In [112]:
driver.close()